In [1]:
import os
#os.chdir('data')
#!datalad install https://github.com/OpenNeuroDatasets/ds000008.git
## https://openneuro.org/datasets/ds000008/versions/00001
#os.chdir('..')

install(ok): U:\ownCloud\ownCloud\fMRI_trainig\ds000008 (dataset)


[INFO] Cloning dataset to Dataset("U:\ownCloud\ownCloud\fMRI_trainig\ds000008") 
[INFO] Attempting to clone from https://github.com/OpenNeuroDatasets/ds000008.git to U:\ownCloud\ownCloud\fMRI_trainig\ds000008 
[INFO] Start enumerating objects 
[INFO] Start counting objects 
[INFO] Start compressing objects 
[INFO] Start receiving objects 
[INFO] Start resolving deltas 
[INFO] Completed clone attempts for Dataset("U:\ownCloud\ownCloud\fMRI_trainig\ds000008") 
[INFO] Detected a filesystem without fifo support. 
[INFO] Disabling ssh connection caching. 
[INFO] Detected a crippled filesystem. 
[INFO] Entering an adjusted branch where files are unlocked as this filesystem does not support locked files. 
[INFO] Updating files:   4% (5/106) 
[INFO] Updating files:   5% (6/106) 
[INFO] Updating files:   6% (7/106) 
[INFO] Updating files:   7% (8/106) 
[INFO] Updating files:   8% (9/106) 
[INFO] Updating files:   9% (10/106) 
[INFO] Updating files:  10% (11/106) 
[INFO] Updating files:  11% (12

In [14]:
from bids.layout import BIDSLayout
data_path = 'data\\ds000007\\'
layout = BIDSLayout(data_path)

In [3]:
import seedir as sd

In [31]:
sd.seedir(data_path, style='emoji')

📁 ds000007/
├─📁 .datalad/
│ ├─📄 .gitattributes
│ └─📄 config
├─📄 .gitattributes
├─📄 annex-uuid
├─📄 CHANGES
├─📄 dataset_description.json
├─📁 derivatives/
│ └─📁 mriqc/
│   ├─📄 aMRIQC.csv
│   ├─📄 anatomical_group.pdf
│   ├─📄 anatomical_sub-01.pdf
│   ├─📄 anatomical_sub-02.pdf
│   ├─📄 anatomical_sub-03.pdf
│   ├─📄 anatomical_sub-04.pdf
│   ├─📄 anatomical_sub-05.pdf
│   ├─📄 anatomical_sub-06.pdf
│   ├─📄 anatomical_sub-07.pdf
│   ├─📄 anatomical_sub-08.pdf
│   ├─📄 anatomical_sub-09.pdf
│   ├─📄 anatomical_sub-10.pdf
│   ├─📄 anatomical_sub-11.pdf
│   ├─📄 anatomical_sub-12.pdf
│   ├─📄 anatomical_sub-13.pdf
│   ├─📄 anatomical_sub-14.pdf
│   ├─📄 anatomical_sub-15.pdf
│   ├─📄 anatomical_sub-16.pdf
│   ├─📄 anatomical_sub-17.pdf
│   ├─📄 anatomical_sub-18.pdf
│   ├─📄 anatomical_sub-19.pdf
│   ├─📄 anatomical_sub-20.pdf
│   ├─📄 fMRIQC.csv
│   ├─📄 functional_group.pdf
│   ├─📄 functional_sub-01.pdf
│   ├─📄 functional_sub-02.pdf
│   ├─📄 functional_sub-03.pdf
│   ├─📄 functional_sub-04.pdf
│   ├─📄 functional_

Get all subject IDs

In [32]:
layout.get_subjects()

['16',
 '20',
 '10',
 '04',
 '14',
 '03',
 '05',
 '06',
 '02',
 '19',
 '11',
 '18',
 '09',
 '01',
 '17',
 '15',
 '13',
 '07',
 '08',
 '12']

In [33]:
layout.get_entities()

{'subject': <bids.layout.models.Entity at 0x22c2ca7fe50>,
 'session': <bids.layout.models.Entity at 0x22c2ca7fe20>,
 'task': <bids.layout.models.Entity at 0x22c2ca7fee0>,
 'acquisition': <bids.layout.models.Entity at 0x22c2c43ce20>,
 'ceagent': <bids.layout.models.Entity at 0x22c2c43cf70>,
 'reconstruction': <bids.layout.models.Entity at 0x22c2c43c9d0>,
 'direction': <bids.layout.models.Entity at 0x22c2c43cd90>,
 'run': <bids.layout.models.Entity at 0x22c2c43c6d0>,
 'proc': <bids.layout.models.Entity at 0x22c2cd48280>,
 'modality': <bids.layout.models.Entity at 0x22c2cdfbfd0>,
 'echo': <bids.layout.models.Entity at 0x22c2cdfbcd0>,
 'flip': <bids.layout.models.Entity at 0x22c2cdfbc70>,
 'inv': <bids.layout.models.Entity at 0x22c2cdfbbe0>,
 'mt': <bids.layout.models.Entity at 0x22c2cdfb3d0>,
 'part': <bids.layout.models.Entity at 0x22c2cd2c7c0>,
 'recording': <bids.layout.models.Entity at 0x22c2cd2c520>,
 'space': <bids.layout.models.Entity at 0x22c2cd2c2e0>,
 'suffix': <bids.layout.mode

In [34]:
# Here we're using an example BIDS dataset that's bundled with the pybids tests


# Initialize the layout
layout = BIDSLayout(data_path)

# Print some basic information about the layout
layout

BIDS Layout: ...oud\fMRI_trainig\data\ds000007 | Subjects: 20 | Sessions: 0 | Runs: 40

In [8]:
layout.get(return_type='filename')[:10]

['D:\\ownCloud\\ownCloud\\fMRI_trainig\\data\\ds000008\\CHANGES',
 'D:\\ownCloud\\ownCloud\\fMRI_trainig\\data\\ds000008\\dataset_description.json',
 'D:\\ownCloud\\ownCloud\\fMRI_trainig\\data\\ds000008\\participants.tsv',
 'D:\\ownCloud\\ownCloud\\fMRI_trainig\\data\\ds000008\\README',
 'D:\\ownCloud\\ownCloud\\fMRI_trainig\\data\\ds000008\\sub-01\\anat\\sub-01_inplaneT2.nii.gz',
 'D:\\ownCloud\\ownCloud\\fMRI_trainig\\data\\ds000008\\sub-01\\anat\\sub-01_T1w.nii.gz',
 'D:\\ownCloud\\ownCloud\\fMRI_trainig\\data\\ds000008\\sub-01\\func\\sub-01_task-conditionalstopsignal_run-01_bold.nii.gz',
 'D:\\ownCloud\\ownCloud\\fMRI_trainig\\data\\ds000008\\sub-01\\func\\sub-01_task-conditionalstopsignal_run-01_events.tsv',
 'D:\\ownCloud\\ownCloud\\fMRI_trainig\\data\\ds000008\\sub-01\\func\\sub-01_task-conditionalstopsignal_run-02_bold.nii.gz',
 'D:\\ownCloud\\ownCloud\\fMRI_trainig\\data\\ds000008\\sub-01\\func\\sub-01_task-conditionalstopsignal_run-02_events.tsv']

In [35]:
layout.get_tasks()

['stopword', 'stopvocal', 'stopmanual']

In [10]:
# Retrieve filenames of all BOLD runs for subject 01
layout.get(subject='02', extension='nii.gz', suffix='bold', return_type='filename')

['D:\\ownCloud\\ownCloud\\fMRI_trainig\\data\\ds000008\\sub-02\\func\\sub-02_task-conditionalstopsignal_run-01_bold.nii.gz',
 'D:\\ownCloud\\ownCloud\\fMRI_trainig\\data\\ds000008\\sub-02\\func\\sub-02_task-conditionalstopsignal_run-02_bold.nii.gz',
 'D:\\ownCloud\\ownCloud\\fMRI_trainig\\data\\ds000008\\sub-02\\func\\sub-02_task-stopsignal_run-01_bold.nii.gz',
 'D:\\ownCloud\\ownCloud\\fMRI_trainig\\data\\ds000008\\sub-02\\func\\sub-02_task-stopsignal_run-02_bold.nii.gz',
 'D:\\ownCloud\\ownCloud\\fMRI_trainig\\data\\ds000008\\sub-02\\func\\sub-02_task-stopsignal_run-03_bold.nii.gz']

In [36]:
from bids.reports import BIDSReport
#from bids.tests import get_test_data_path
#layout = BIDSLayout(os.path.join(get_test_data_path(), 'synthetic'))

report = BIDSReport(layout)

counter = report.generate()

AttributeError: 'list' object has no attribute 'split'

In [12]:
main_report = counter.most_common()[0][0]
print(main_report)

For session None:
	MR data were acquired using a UNKNOWN-Tesla MANUFACTURER MODEL MRI scanner.
	Three runs of conditional stop signal UNKNOWN-echo fMRI data were collected (30 slices; repetition time, TR=2000ms; echo time, TE=UNKNOWNms; flip angle, FA=UNKNOWN<deg>; field of view, FOV=200x200mm; matrix size=64x64; voxel size=3.12x3.12x5mm). Each run was 5:52 minutes in length, during which 176 functional volumes were acquired. 
	Three runs of stop signal UNKNOWN-echo fMRI data were collected (30 slices; repetition time, TR=2000ms; echo time, TE=UNKNOWNms; flip angle, FA=UNKNOWN<deg>; field of view, FOV=200x200mm; matrix size=64x64; voxel size=3.12x3.12x5mm). Each run was 6:04 minutes in length, during which 182 functional volumes were acquired. 

Dicoms were converted to NIfTI-1 format. This section was (in part) generated automatically using pybids (0.14.0). 


In [19]:
# Convert the layout to a pandas dataframe
df = layout.to_df()
df.head(10)

entity,path,datatype,extension,run,subject,suffix,task
0,D:\ownCloud\ownCloud\fMRI_trainig\data\ds00000...,NaN,.json,NaN,NaN,T1w,NaN
1,D:\ownCloud\ownCloud\fMRI_trainig\data\ds00000...,NaN,.json,NaN,NaN,description,NaN
2,D:\ownCloud\ownCloud\fMRI_trainig\data\ds00000...,NaN,.json,NaN,NaN,inplaneT2,NaN
3,D:\ownCloud\ownCloud\fMRI_trainig\data\ds00000...,NaN,.tsv,NaN,NaN,participants,NaN
4,D:\ownCloud\ownCloud\fMRI_trainig\data\ds00000...,anat,.json,NaN,01,T1w,NaN
5,D:\ownCloud\ownCloud\fMRI_trainig\data\ds00000...,anat,.nii.gz,NaN,01,T1w,NaN
6,D:\ownCloud\ownCloud\fMRI_trainig\data\ds00000...,anat,.json,NaN,01,inplaneT2,NaN
7,D:\ownCloud\ownCloud\fMRI_trainig\data\ds00000...,anat,.nii.gz,NaN,01,inplaneT2,NaN
8,D:\ownCloud\ownCloud\fMRI_trainig\data\ds00000...,func,.json,1,01,bold,stopmanual
9,D:\ownCloud\ownCloud\fMRI_trainig\data\ds00000...,func,.nii.gz,1,01,bold,stopmanual


In [20]:
layout.to_df(metadata=True).head(10)

entity,path,BandwidthPerPixelPhaseEncode,Columns,EchoTime,EchoTrainLength,FlipAngle,ImageType,ImagingFrequency,InplanePhaseEncodingDirection,InversionTime,...,SpacingBetweenSlices,StudyID,StudyTime,TaskName,datatype,extension,run,subject,suffix,task
0,D:\ownCloud\ownCloud\fMRI_trainig\data\ds00000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,.json,NaN,NaN,T1w,NaN
1,D:\ownCloud\ownCloud\fMRI_trainig\data\ds00000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,.json,NaN,NaN,description,NaN
2,D:\ownCloud\ownCloud\fMRI_trainig\data\ds00000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,.json,NaN,NaN,inplaneT2,NaN
3,D:\ownCloud\ownCloud\fMRI_trainig\data\ds00000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,.tsv,NaN,NaN,participants,NaN
4,D:\ownCloud\ownCloud\fMRI_trainig\data\ds00000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,anat,.json,NaN,01,T1w,NaN
5,D:\ownCloud\ownCloud\fMRI_trainig\data\ds00000...,NaN,192,0.0021,1,8.0,ORIGINAL/PRIMARY/M/ND,123.253116,ROW,1.1,...,NaN,1,160330.968000,NaN,anat,.nii.gz,NaN,01,T1w,NaN
6,D:\ownCloud\ownCloud\fMRI_trainig\data\ds00000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,anat,.json,NaN,01,inplaneT2,NaN
7,D:\ownCloud\ownCloud\fMRI_trainig\data\ds00000...,NaN,128,0.033,1,90.0,ORIGINAL/PRIMARY/M/ND,123.253096,COL,0.0,...,4.0,1,160330.968000,NaN,anat,.nii.gz,NaN,01,inplaneT2,NaN
8,D:\ownCloud\ownCloud\fMRI_trainig\data\ds00000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,func,.json,1,01,bold,stopmanual
9,D:\ownCloud\ownCloud\fMRI_trainig\data\ds00000...,NaN,384,0.03,1,90.0,ORIGINAL/PRIMARY/M/ND/MOSAIC,123.253096,COL,0.0,...,4.0,1,160330.968000,Stop Manual,func,.nii.gz,1,01,bold,stopmanual


In [25]:
# !cat on unix
!type data\\ds000007\\sub-01\\func\\sub-01_task-stopmanual_run-1_bold.json

{
   "Columns": 384, 
   "EchoTime": 0.03, 
   "EchoTrainLength": 1, 
   "FlipAngle": 90.0, 
   "ImageType": "ORIGINAL/PRIMARY/M/ND/MOSAIC", 
   "ImagingFrequency": 123.253096, 
   "InplanePhaseEncodingDirection": "COL", 
   "InversionTime": 0.0, 
   "MRAcquisitionType": "2D", 
   "MagneticFieldStrength": 2.8936200141907, 
   "Manufacturer": "SIEMENS", 
   "ManufacturersModelName": "Allegra", 
   "NumberOfAverages": 1.0, 
   "NumberOfPhaseEncodingSteps": 64, 
   "PercentPhaseFieldOfView": 100.0, 
   "PhaseEncodingDirection": "j-", 
   "PixelBandwidth": 2605.0, 
   "PixelSpacing": [
      3.125, 
      3.125
   ], 
   "RepetitionTime": 2, 
   "Rows": 384, 
   "ScanOptions": "FS", 
   "ScanningSequence": [
      "SE", 
      "EP"
   ], 
   "SequenceName": "*epfid2d1_64", 
   "SequenceVariant": "SK", 
   "SeriesDate": "18660506", 
   "SeriesDescription": "BOLD EPI - Stopmanual1", 
   "SeriesNumber": 5, 
   "SeriesTime": "162243.859000", 
   "SliceThickness": 4.0, 
   "SpacingBetweenSlices

In [26]:
# !cat on unix
!type data\\ds000007\\sub-01\\anat\\sub-01_T1w.json

{
   "Columns": 192, 
   "EchoTime": 0.0021, 
   "EchoTrainLength": 1, 
   "FlipAngle": 8.0, 
   "ImageType": "ORIGINAL/PRIMARY/M/ND", 
   "ImagingFrequency": 123.253116, 
   "InplanePhaseEncodingDirection": "ROW", 
   "InversionTime": 1.1, 
   "MRAcquisitionType": "3D", 
   "MagneticFieldStrength": 2.8936200141907, 
   "Manufacturer": "SIEMENS", 
   "ManufacturersModelName": "Allegra", 
   "NumberOfAverages": 1.0, 
   "NumberOfPhaseEncodingSteps": 192, 
   "PercentPhaseFieldOfView": 100.0, 
   "PhaseEncodingDirection": "i-", 
   "PixelBandwidth": 320.0, 
   "PixelSpacing": [
      1.3333333333333, 
      1.3333333333333
   ], 
   "RepetitionTime": 2.3, 
   "Rows": 192, 
   "ScanOptions": "IR", 
   "ScanningSequence": [
      "IR", 
      "GR"
   ], 
   "SequenceName": "*tfl3d1", 
   "SequenceVariant": [
      "SP", 
      "MP"
   ], 
   "SeriesDate": "18660506", 
   "SeriesDescription": "MPRage-3T", 
   "SeriesNumber": 11, 
   "SeriesTime": "171732.437000", 
   "SliceThickness": 1.0, 